In [1]:
import time

import torch
import torchvision

In [2]:
# ResNet50 の準備
model = torchvision.models.resnet50()
model = model.to("cuda")
model.eval()

# ダミー画像を用意
input_image = torch.ones((32, 3, 224, 224))
input_image = input_image.to("cuda")

# ウォームアップのため一度走らせる
with torch.no_grad():
    output = model(input_image)

# モデルの計算時間を計測（FP 32）
torch.cuda.synchronize()
start_time = time.time()
with torch.no_grad():
    output = model(input_image)
torch.cuda.synchronize()
end_time = time.time()

computation_time = end_time - start_time
print(computation_time)
# -> RTX 3060 GPU で 0.057 秒

0.05667281150817871


In [3]:
# モデルを torch.compile に変換
model_opt = torch.compile(model, mode="max-autotune")

with torch.no_grad():
    for i in range(10):
        output = model_opt(input_image)

# モデルの計算時間を計測（torch.compie）
torch.cuda.synchronize()
start_time = time.time()
with torch.no_grad():
    output = model_opt(input_image)
torch.cuda.synchronize()
end_time = time.time()

computation_time = end_time - start_time
print(computation_time)
# -> RTX 3060 GPU で 0.043 秒

/home/joisino/.cache/pypoetry/virtualenvs/speedbook-5IKDUY1E-py3.11/lib/python3.11/site-packages/torch/_inductor/compile_fx.py:124: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(
W0816 16:14:04.163000 132088678411136 torch/_inductor/utils.py:945] [0/0] not enough SMs to use max_autotune_gemm mode
AUTOTUNE addmm(32x1000, 32x2048, 2048x1000)
  bias_addmm 0.0686 ms 100.0%
  addmm 0.1055 ms 65.0%
SingleProcess AUTOTUNE takes 0.2273 seconds


0.04272150993347168
